In [1]:
# -
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

from scipy.io import loadmat
from scipy.signal import hilbert, welch

from scipy.signal import butter, lfilter
from fakespikes.util import create_times
from collections import defaultdict

# -
import os
import h5py
import numpy as np

from tqdm import tqdm
from glob import glob
from pprint import pprint

from scipy.io import loadmat

from fakespikes.util import create_times
from fakespikes.util import create_psd

from alphalogical.util import butter_bandpass_filter
from alphalogical.util import hilbert_power
from alphalogical.util import load_good_smith_files
from alphalogical.util import load_smith_foof_results
from alphalogical.util import load_smith_rates
from alphalogical.util import load_smith_lfps

from alphalogical.util import find_bursts
from alphalogical.util import find_extrema
from alphalogical.util import analyze_bursts

Loading BokehJS ...

# Pathing

In [2]:
DATA_PATH = "/Users/type/Data/Smith/data"


# Load data and segment files

In [3]:
lfp_files, rate_files, seg_files = load_good_smith_files(DATA_PATH)

In [4]:
pprint([(i, fi) for i, fi in enumerate(lfp_files)])

[(0,
  '/Users/type/Data/Smith/data/Bo130408_s6ae_fixblank_active_0001_converted_ns2.mat'),
 (1,
  '/Users/type/Data/Smith/data/Bo130408_s6ae_fixblank_active_0002_converted_ns2.mat'),
 (2,
  '/Users/type/Data/Smith/data/Bo130409_s7ae_fixblank_active_0003_converted_ns2.mat'),
 (3,
  '/Users/type/Data/Smith/data/Wi130116_s51ae_fixblank_active_0001_converted_ns2.mat'),
 (4,
  '/Users/type/Data/Smith/data/Bo130404_s4ae_fixblank_active_0002_converted_ns2.mat'),
 (5,
  '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0001_converted_ns2.mat'),
 (6,
  '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0002_converted_ns2.mat'),
 (7,
  '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0003_converted_ns2.mat'),
 (8,
  '/Users/type/Data/Smith/data/Bo130418_s12ae_fixblank_active_0002_converted_ns2.mat'),
 (9,
  '/Users/type/Data/Smith/data/Wi121219_s43ae_fixblank_active_0001_converted_ns2.mat'),
 (10,
  '/Users/type/Data/Smith/data/Wi121219_s43ae_fixblank_active_0002_conv

In [5]:
pprint([(i, fi) for i, fi in enumerate(rate_files)])

[(0, '/Users/type/Data/Smith/data/Bo130408_s6ae_fixblank_active_0001.mat'),
 (1, '/Users/type/Data/Smith/data/Bo130408_s6ae_fixblank_active_0002.mat'),
 (2, '/Users/type/Data/Smith/data/Bo130409_s7ae_fixblank_active_0003.mat'),
 (3, '/Users/type/Data/Smith/data/Wi130116_s51ae_fixblank_active_0001.mat'),
 (4, '/Users/type/Data/Smith/data/Bo130404_s4ae_fixblank_active_0002.mat'),
 (5, '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0001.mat'),
 (6, '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0002.mat'),
 (7, '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0003.mat'),
 (8, '/Users/type/Data/Smith/data/Bo130418_s12ae_fixblank_active_0002.mat'),
 (9, '/Users/type/Data/Smith/data/Wi121219_s43ae_fixblank_active_0001.mat'),
 (10, '/Users/type/Data/Smith/data/Wi121219_s43ae_fixblank_active_0002.mat'),
 (11, '/Users/type/Data/Smith/data/Wi130129_s55ae_fixblank_active_0001.mat'),
 (12, '/Users/type/Data/Smith/data/Wi130129_s55ae_fixblank_active_0002.mat'),
 (1

In [6]:
pprint([(i, fi) for i, fi in enumerate(rate_files)])

[(0, '/Users/type/Data/Smith/data/Bo130408_s6ae_fixblank_active_0001.mat'),
 (1, '/Users/type/Data/Smith/data/Bo130408_s6ae_fixblank_active_0002.mat'),
 (2, '/Users/type/Data/Smith/data/Bo130409_s7ae_fixblank_active_0003.mat'),
 (3, '/Users/type/Data/Smith/data/Wi130116_s51ae_fixblank_active_0001.mat'),
 (4, '/Users/type/Data/Smith/data/Bo130404_s4ae_fixblank_active_0002.mat'),
 (5, '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0001.mat'),
 (6, '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0002.mat'),
 (7, '/Users/type/Data/Smith/data/Bo130405_s5ae_fixblank_active_0003.mat'),
 (8, '/Users/type/Data/Smith/data/Bo130418_s12ae_fixblank_active_0002.mat'),
 (9, '/Users/type/Data/Smith/data/Wi121219_s43ae_fixblank_active_0001.mat'),
 (10, '/Users/type/Data/Smith/data/Wi121219_s43ae_fixblank_active_0002.mat'),
 (11, '/Users/type/Data/Smith/data/Wi130129_s55ae_fixblank_active_0001.mat'),
 (12, '/Users/type/Data/Smith/data/Wi130129_s55ae_fixblank_active_0002.mat'),
 (1

# Pick the dataset to analyze 

Name it `x`.

In [7]:
n = 0

In [8]:
# Load all its segments
header, segments = load_smith_foof_results(seg_files[n])

print("Header {}".format(header))
print("Segment N: {}".format(len(segments)))
print("Segment row {}".format(segments[0]))

Header ['m', 'c', 'i', 'j', 'center', 'power', 'bw']
Segment N: 11805
Segment row (0, 49, 1131712, 1141712, 13.0381994977, 0.3680282355, 2.83154123279)


# Pick a segment

In [9]:
m = 2004

In [10]:
# Get indices
m, c, i, j, center, power, bw = segments[m]
print(m, i, j)

(2004, 3163175, 3173175)


# Load data for channel `c`

In [11]:
# Get the LFP
fs, times, lfp = load_smith_lfps(lfp_files[n], c)

# Parse time
T = times.max()
times_m = times[i:j]

print("T: {}".format(T))
print("ref shape: {}".format(lfp.shape))
print("lfp_m shape: {}".format(lfp.shape))

T: 3486.306
ref shape: (3486306,)
lfp_m shape: (3486306,)


In [12]:
# get the rate
_, rate = load_smith_rates(rate_files[n], c, (0, T), 1/fs)

print("Rate shape: {}".format(rate.shape))

Rate shape: (3486305,)


## Explore raw timeseries

In [ ]:
# Intial window (s)
x_burst = lfp[i:j]

t = float(x_burst.size / fs)
win = (0, t)

p = figure(plot_width=600, plot_height=150)
p.line(times_m, x_burst, color='black', line_width=1)
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "LFP (mV)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

freqs, psd = welch(x_burst, fs=fs, window='hanning', nperseg=fs)

p = figure(plot_width=250, plot_height=150)
p.line(freqs, psd, color='black', alpha=0.9, line_width=2);
p.x_range = Range1d(0, 50)
p.xaxis.axis_label = "Freq (Hz)"
p.yaxis.axis_label = "Power (AU)"
show(p)

In [ ]:
p = figure(plot_width=600, plot_height=150)
p.line(times_m, rate[i:j], color='black')
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "Rate (Hz)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Find the bursts in m

In [13]:
# --------------------------------------------------------------
verbose = True
m, c, i, j, center, power, bw = segments[m]


# --------------------------------------------------------------
# Do filtering over the whole electrode, for consistency
bw = 2 # Override

if verbose:
    print(">>> Calc. alpha power.")
alpha_range = (center - bw, center + bw)
alpha = butter_bandpass_filter(lfp, alpha_range[0], alpha_range[1], fs, order=2)
alpha_pow = hilbert_power(alpha)

if verbose:
    print(">>> Calc. HG power.")
hg = butter_bandpass_filter(lfp, 100, 150, fs, order=2)
hg_pow = hilbert_power(hg)


# --------------------------------------------------------------
if verbose:
    print(">>> Est. burst threshold.")

# Find low
low_range = [center - bw + 2, center - bw]
if low_range[0] < 0:
    low_range[0] = 1
if low_range[1] < 2:
    low_range[1] = 2
low = butter_bandpass_filter(lfp, low_range[0], low_range[1], fs, order=2)
low = hilbert_power(low)

# Find high
high_range = [center + bw, center + bw + 2]
high = butter_bandpass_filter(lfp, high_range[0], high_range[1], fs, order=2)
high = hilbert_power(high)

# Find extrema for all alpha
if verbose:
    print(">>> Finding peaks.")
peaks, troughs = find_extrema(
    alpha, fs, alpha_range, boundary=None, first_extrema='peak')


# --------------------------------------------------------------
if verbose:
    print(">>> Est. burst threshold.")

# Only run for this mth segment...
M = np.mean((low + high) / 2.0)

if verbose:
    print(">>> Finding bursts")
bursts = find_bursts(alpha_pow[i:j], 3 * M, 1.5 * M)

if verbose:
    print(">>> Finding peaks/troughs in the bursts.")

peaks_b = []
troughs_b = []
for b in bursts:
    b = np.asarray(b)
    
    mask = np.logical_and(peaks >= b.min(), peaks < b.max())
    p_mask = peaks[mask]
    peaks_b.append(p_mask)
    
    mask = np.logical_and(troughs >= b.min(), troughs < b.max())
    t_mask = troughs[mask]
    troughs_b.append(t_mask)

peaks_b = np.concatenate(peaks_b)
troughs_b = np.concatenate(troughs_b)


# --------------------------------------------------------------
if verbose:
    print(">>> Analyzing bursts.")
    
data = alpha_pow  # tmp
burst_results = analyze_bursts(bursts, data, shift=i)

if verbose:
    print(">>> Analyzing peaks/troughs.")

peak_results = analyze_extrema(peaks_b, data, n_sample=5, shift=None)
trough_results = analyze_extrema(troughs_b, data, n_sample=5, shift=None)

>>> Calc. alpha power.
>>> Calc. HG power.
>>> Est. burst threshold.
>>> Finding peaks.
>>> Est. burst threshold.
>>> Finding bursts
>>> Finding peaks/troughs in the bursts.


ValueError: need at least one array to concatenate

In [18]:
rate.shape, np.pad(rate, (1, 0), 'constant').shape

((3486305,), (3486306,))

# Plot the threshold and the bursts

### LFP

In [ ]:
x_filt = butter_bandpass_filter(lfp[i:j], 3, 100, fs, order=2)

p = figure(plot_width=800, plot_height=250)
p.line(times[i:j], x_filt[i:j], color='black', alpha=0.3)
p.line(times[i:j], alpha[i:j], color='red', alpha=0.6)
p.line(times[i:j], alpha_pow[i:j], color='red', line_width=5, alpha=0.2)
p.line(times[i:j], M, color='gold', line_dash="dashed", line_width=5)
for b in bursts:
    start, stop = min(b), max(b)
    p.line(x=times[i:j][start:stop], y=x_filt.max(), color="blue", line_width=5, alpha=0.2)
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "Power (AU)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

### Rate

In [ ]:
p = figure(plot_width=800, plot_height=250)
p.line(times_m, rate[i:j], color='black', alpha=0.3)
for b in bursts:
    start, stop = min(b), max(b)
    p.line(x=times_m[start:stop], y=rate.max(), color="blue", line_width=5, alpha=0.2)
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "Power (AU)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Analyze bursts

In [ ]:
peaks, troughs = find_extrema(
        alpha[i:j],
        fs, (center - 2, center + 2),
        boundary=None,
        first_extrema='peak')

In [ ]:
alpha

In [ ]:
zerofallN

In [ ]:
results = analyze_bursts(m, bursts, alpha_pow, fs=fs)
pprint(results)

# Analyze all segments for file `n`

In [ ]:
from joblib import Parallel, delayed

def run(lfp_filename, rate_filename, row):
    # Segment data
    m, c, i, j, center, power, bw = row

    # Load lfp
    fs, times, x_ref = load_smith_lfps(lfp_filename, c)

    # Parse time
    T = times.max()

    # load firing rate
    _, x_rate_ref = load_smith_rates(rate_filename, c, (0, T), 1/fs)

    # Get segment data only
    times_m = times[i:j]
    x_lfp = x_ref[i:j]
    x_rate = x_rate_ref[i:j]

    # Burst analysis
    bursts, alpha, alpha_pow, M = find_smith_bursts(x_ref, x_lfp, center, fs)
    result = analyze_smith_bursts(m, bursts, alpha_pow, hg_pow, x_rate, fs)

    return result

# Plot distributions of burst statistics

- TODO adapt below

### Burst distributions

In [ ]:
hist, edges = np.histogram(bursts["L"], density=False, bins=100)
p = figure(plot_width=300, plot_height=200)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], color="black")
p.xaxis.axis_label = "L (s)"
p.yaxis.axis_label = "Count"
show(p)

### Alpha

In [ ]:
hist, edges = np.histogram(bursts["alpha"], density=False, bins=300)
p = figure(plot_width=300, plot_height=200)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], color="black")
p.xaxis.axis_label = "Alpha power"
p.yaxis.axis_label = "Count"
show(p)

In [ ]:
hist1, edges1 = np.histogram(bursts["alpha_burst"], density=True, bins=50)
hist2, edges2 = np.histogram(bursts["alpha_osc"], density=True, bins=50)

p = figure(plot_width=300, plot_height=200)
p.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:], color="blue", legend="Burst", alpha=0.3)
p.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:], color="purple", legend="Osc.", alpha=0.3)
p.xaxis.axis_label = "Alpha power"
p.yaxis.axis_label = "Count"
show(p)

### Gamma

In [ ]:
hist, edges = np.histogram(bursts["hg"], density=False, bins=300)
p = figure(plot_width=300, plot_height=200)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], color="black")
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Count"
show(p)

In [ ]:
hist1, edges1 = np.histogram(bursts["hg_burst"], density=True, bins=50)
hist2, edges2 = np.histogram(bursts["hg_osc"], density=True, bins=50)

p = figure(plot_width=300, plot_height=200)
p.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:], color="blue", legend="Burst", alpha=0.3)
p.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:], color="purple", legend="Osc.", alpha=0.3)
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Count"
show(p)

### Alpha by HG power

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(bursts["hg"], bursts["alpha"], color='black', alpha=0.3)
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Alpha power"
show(p)

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(bursts["hg_burst"], bursts["alpha_burst"], color='blue', alpha=0.3, legend="Burst")
p.circle(bursts["hg_osc"], bursts["alpha_osc"], color='purple', alpha=0.3, legend="Osc.")
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Alpha power"
show(p)

### L by power

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(burst_lens, burst_hgpower, color='black', alpha=0.3)
p.xaxis.axis_label = "Length (s)"
p.yaxis.axis_label = "Gamma power"
show(p)

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(burst_lens, burst_alphapower, color='black')
p.xaxis.axis_label = "Length (s)"
p.yaxis.axis_label = "Alphs power"
show(p)

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(burst_lens[burst_mask], burst_alphapower[burst_mask], color='blue', alpha=0.3, legend="Burst")
p.circle(burst_lens[osc_mask], burst_alphapower[osc_mask], color='purple', alpha=0.3, legend="Osc")
p.xaxis.axis_label = "Length (s)"
p.yaxis.axis_label = "Alphs power"
show(p)